In [3]:
import spacy
import nltk
from nltk.stem import WordNetLemmatizer

In [4]:
with open('tweet') as data_file:
        sentences = data_file.readlines()

In [5]:
nlp = spacy.load('en_core_web_lg')
wordnet_lemmatizer = WordNetLemmatizer() 

In [6]:

SPACY_DEP_ROOT = "ROOT"
SPACY_DEP_NSUBJ = "nsubj"
SPACY_DEP_NSUBJ_PASS = "nsubjpass"
SPACY_DEP_DOBJ = "dobj"
SPACY_DEP_PREP = "prep"
SPACY_DEP_PREP_OBJ = "pobj"
SPACY_DEP_CCOMP = "ccomp"
SPACY_DEP_IND_OBJ_1 = "dative"
SPACY_DEP_IND_OBJ_2 = "iobj"

SPACY_TAG_PREP = "IN"
SPACY_TAG_NN = "NN"
SPACY_TAG_NNS = "NNS"
SPACY_TAG_NNP = "NNP"
SPACY_TAG_NNPS = "NNPS"
SPACY_TAG_JJ = "JJ"
SPACY_TAG_JJR = "JJR"
SPACY_TAG_JJS = "JJS"
SPACY_TAG_VB = "VB"
SPACY_TAG_VBD = "VBD"
SPACY_TAG_VBG = "VBG"
SPACY_TAG_VBN = "VBN"
SPACY_TAG_VBP = "VBP"
SPACY_TAG_VBZ = "VBZ"

PREP_FOR = "for"
PREP_TO = "to"
PREP_FROM = "from"
PREP_OVER = "over"
PREP_UNTIL = "until"

def getSubTreeString(token):
    return ' '.join([str(token1.text) for token1 in list(token.subtree)])

def getHeadOfSentence(sentence):

    doc = nlp(str(sentence))
    head = None
    for token in doc:
        if token.dep_ == SPACY_DEP_ROOT:
            #print ("head",sentence, token.text)
            head = token
            
    return head  

def printTree(sentence):
    doc = nlp(str(sentence))
 
    for token in doc:
        print("{5}: {0}/{1} <--{2}-- {3}/{4} > {6};  type {7}".format(
           token.text, token.tag_, token.dep_, token.head.text, token.head.tag_,token.i,token.head.i, token.ent_id_))
        
        print(list(token.subtree))
        print ('**********')

In [18]:

    
def isTypeBan(sentence):
    pass

def parse(sentence):
    pass
    
def parseNounForm(sentence):
    timePeriod = None
    reason = None
    who = None
    fromWhat = None
    associatedPrepositionIds = {}
    doc = nlp(str(sentence))
    headToken = getHeadOfSentence(sentence)
    
    
    for token in doc:       
        tokenLemma = wordnet_lemmatizer.lemmatize(token.text)
        if tokenLemma == 'ban' or tokenLemma == 'banned':
            banToken = token
        elif token.head.tag_ in [SPACY_TAG_VBN] and token.head.i == headToken.i and (token.dep_ in [SPACY_DEP_NSUBJ_PASS, SPACY_DEP_NSUBJ]):
            who = getSubTreeString(token)
        elif token.head.tag_ != SPACY_TAG_VBN and token.head.i == headToken.i and (token.dep_ in [SPACY_DEP_IND_OBJ_1, SPACY_DEP_IND_OBJ_2]):
            who = getSubTreeString(token)
    # Fetching preps connected to Ban tag
    for token in doc:
        if token.head.i == banToken.i and token.tag_ == SPACY_TAG_PREP:
            associatedPrepositionIds[token.i] = token
        
    for token in doc:
        if token.head.i in associatedPrepositionIds:
            prep = associatedPrepositionIds[token.head.i]
            if prep.text == PREP_FOR :
                if token.tag_ in [SPACY_TAG_NN, SPACY_TAG_NNS, SPACY_TAG_NNP, SPACY_TAG_NNPS]:     
                    timePeriod = getSubTreeString(token)
                else:
                    reason = getSubTreeString(token)
            elif prep.text == PREP_FROM:
                fromWhat = getSubTreeString(token)
            elif prep.text == PREP_OVER:
                reason = getSubTreeString(token)
            elif prep.text == PREP_UNTIL:
                timePeriod = getSubTreeString(token)
            elif prep.text == PREP_TO:
                who = getSubTreeString(token)
        elif token.head.i == banToken.i and token.tag_ in [SPACY_TAG_NN, SPACY_TAG_NNS, SPACY_TAG_NNP, SPACY_TAG_NNPS, SPACY_TAG_JJ,SPACY_TAG_JJR, SPACY_TAG_JJS]:
            timePeriod = getSubTreeString(token)
    
    print(sentence)
    print(" who: ", who,
         " reason :", reason,
         " timePeriod :", timePeriod,"fromWhat:", fromWhat,'\n')
    
    
    

In [25]:
for sentence in sentences:
    sentence = sentence.replace('tweets','says')
    sentence = sentence.replace('\n','')
    #print(sentence)
    parseVerbForm(sentence)

Dhaniya lana bhul gaya, says user on Australian pacer's viral pic
Who:  user ; Tweet:  Dhaniya lana bhul gaya,  Context:  on Australian pacer 's viral pic 

Dhoni knew about Rafale, says user on Dhoni's T20I exclusion
Who:  user ; Tweet:  Dhoni knew about Rafale,  Context:  on Dhoni 's T20I exclusion 

Chatting to girlfriend after long, says user on Ali's run-out
Who:  user ; Tweet:  Chatting to girlfriend after long,  Context:  on Ali 's run - out 

Keeps the bowler guessing, says user on Sarfraz's stance
Who:  user ; Tweet:  Keeps the bowler guessing,  Context:  on Sarfraz 's stance 

Aajao duo khelte hai, says user on Dhoni's tweet about PUBG
Who:  hai ; Tweet:  ,user on Dhoni 's tweet about PUBG  Context:  None 

English FA facing sexism claims over women's team tweet
Who:  claims ; Tweet:    Context:  over women 's team 

Didn't see these cocky says vs England: Ex-WI pacer to Harbhajan
Who:   ; Tweet:  Didn'tthese cocky says vs England :Ex - WI pacer to Harbhajan  Context:  None 


In [24]:
sentence = "Nobody does it better, says user as Dhoni plots Shakib's wicket"
#sentences = nltk.sent_tokenize(sentence)
print(sentence)
printTree(sentence)
parseVerbForm(sentence)

Nobody does it better, says user as Dhoni plots Shakib's wicket
0: Nobody/NN <--nsubj-- does/VBZ > 1;  type 
[Nobody]
**********
1: does/VBZ <--ccomp-- says/VBZ > 5;  type 
[Nobody, does, it, better]
**********
2: it/PRP <--dobj-- does/VBZ > 1;  type 
[it]
**********
3: better/RBR <--advmod-- does/VBZ > 1;  type 
[better]
**********
4: ,/, <--punct-- says/VBZ > 5;  type 
[,]
**********
5: says/VBZ <--ROOT-- says/VBZ > 5;  type 
[Nobody, does, it, better, ,, says, user, as, Dhoni, plots, Shakib, 's, wicket]
**********
6: user/NN <--nsubj-- says/VBZ > 5;  type 
[user]
**********
7: as/IN <--mark-- plots/VBZ > 9;  type 
[as]
**********
8: Dhoni/NNP <--nsubj-- plots/VBZ > 9;  type 
[Dhoni]
**********
9: plots/VBZ <--advcl-- says/VBZ > 5;  type 
[as, Dhoni, plots, Shakib, 's, wicket]
**********
10: Shakib/NNP <--poss-- wicket/NN > 12;  type 
[Shakib, 's]
**********
11: 's/POS <--case-- Shakib/NNP > 10;  type 
['s]
**********
12: wicket/NN <--dobj-- plots/VBZ > 9;  type 
[Shakib, 's, wicket]

In [22]:
def parseVerbForm(sentence):
    who = ""
    whoToken = None
    tweet = ""
    tweetToken = None
    context = None
    
    saysToken = getHeadOfSentence(sentence)
    doc = nlp(sentence)
    for token in doc:
        if token.head.i == saysToken.i and token.i != saysToken.i: 
            #print(token.text)
            if token.dep_ == SPACY_DEP_NSUBJ:
                who = (token.text)
                whoToken = token
            else:
                tweet += getSubTreeString(token)
                
    if whoToken != None:
        for token in doc:
            if token.tag_ == SPACY_TAG_PREP and token.head.i == whoToken.i:
                context = getSubTreeString(token)
    print(sentence)           
    print("Who: ",who,"; Tweet: ",tweet," Context: ", context,'\n')

In [23]:
sentence = "An incredible week for Sharmas, says user on Rohit's 208"
parseVerbForm(sentence)

An incredible week for Sharmas, says user on Rohit's 208
Who:  user ; Tweet:  An incredible week for Sharmas,  Context:  on Rohit 's 208 



In [ ]:
#Add subject of the tweet